# 1. Serializacion

- Procedimiento de transformar un objeto en una secuencia de *bytes*
- Util para almacenar el estado del objeto en un archivo o ser enviado a otros lados

## `pickle`

Modulo para guardar y cargar objetos
- `dumps`: Serializa -> Guarda un objeto
- `loads`: Deserializa (un objeto serializado) -> Carga un objeto a su estado original (se crea uno nuevo **identico pero internamente** distinto al original). Hace una copia

In [8]:
import pickle

# tupla_names = ('Daniel', 'Walter', 'Paola', 2, 'Tintin')
tupla = (1, 2, 3, 4, 5, 6, 7, 8, 9)
tupla_serializada = pickle.dumps(tupla)

print(f'Tupla serializada: {tupla_serializada}')
print(f'Tipo de la serializacion: {type(tupla_serializada)}')

print()

tupla_deserializada = pickle.loads(tupla_serializada)
print(f'Tupla deserializada: {tupla_deserializada}')
print(f'Tupla original: {tupla}')

print()

print(f'¿Son iguales entre si?: {tupla == tupla_deserializada}') # son iguales
print(f'¿Son el mismo objeto?: {tupla is tupla_deserializada}') # no son el mismo objeto

Tupla serializada: b'\x80\x04\x95\x16\x00\x00\x00\x00\x00\x00\x00(K\x01K\x02K\x03K\x04K\x05K\x06K\x07K\x08K\tt\x94.'
Tipo de la serializacion: <class 'bytes'>

Tupla deserializada: (1, 2, 3, 4, 5, 6, 7, 8, 9)
Tupla original: (1, 2, 3, 4, 5, 6, 7, 8, 9)

¿Son iguales entre si?: True
¿Son el mismo objeto?: False


### Serializacion de archivos
- `dump(<objeto>, <file>)`: serializa un archivo
- `load`: deserializa un arhivo serializado, se crea un nuevo objeto **distinto al original**. No seguro cargar archivos a traves de este modulo

- se especifia si se van a escribit bytes (`wb`) o leer bytes (`rb`)

In [18]:
from os import path

# creamos una lista que sera almacenada en un archivo

lista = [1, 2, 3, 4, 5]

# se crea el archivo 'mi_lista.bin'
with open(path.join('mi_lista.bin'), 'wb') as arch:
    pickle.dump(lista, arch)


with open(path.join('mi_lista.bin'), 'rb') as arch:
    lista_cargada = pickle.load(arch)

print(f'Lista og: {lista}')
print(f'Lista deserializada: {lista_cargada}')
print(f'¿Son iguales?: {lista_cargada == lista}')
print(f'¿Son el mismo objeto? {lista_cargada is lista}')

Lista og: [1, 2, 3, 4, 5]
Lista deserializada: [1, 2, 3, 4, 5]
¿Son iguales?: True
¿Son el mismo objeto? False


### Personalizar serializacion - Metodo `__getstate__`

Cuando se serializa un objeto `pickle` busca que este implementado el metodo `__getstate__`.

- Debe retornar un diccionario con los atributos a serializar

- Si **no** esta implementado: por defecto `pickle` guarda el atributo `__dict__` del objeto (diccionario que guarda todos los metodos y atributos de un objeto)

In [28]:
class Perro:

    def __init__(self, nombre, raza) -> None:
        self.nombre = nombre
        self.raza = raza
        self.mensaje = "Atributo normal"

    # implementamos __getitem__

    def __getstate__(self):

        # copiamos el diccionario de metodos y atributos
        nuevo = self.__dict__.copy()

        # modificamos uno de los atributos para la serializacion
        nuevo.update({"mensaje": "Atributo serializado"})

        # lo que se retorna es el resultado de la serializion
        return nuevo
    
    def ladrar(self):
        print('guau')

print(Perro.__dict__)

{'__module__': '__main__', '__init__': <function Perro.__init__ at 0x7f5738ebb9a0>, '__getstate__': <function Perro.__getstate__ at 0x7f5738ebb910>, 'ladrar': <function Perro.ladrar at 0x7f5738ebb7f0>, '__dict__': <attribute '__dict__' of 'Perro' objects>, '__weakref__': <attribute '__weakref__' of 'Perro' objects>, '__doc__': None}


In [29]:
perro = Perro('TinTin', 'Caniche')
print(perro.__dict__)

print(f'Original: {perro.mensaje}')

# serializamos y deserializamos

perro_ser = pickle.dumps(perro)
perro_deser = pickle.loads(perro_ser)

print('\nComprobacion Resultados:')
print(f'Mensaje original: {perro.mensaje}') # el perro sigue igual
print(f'Mensaje deserializado: {perro_deser.mensaje}')

{'nombre': 'TinTin', 'raza': 'Caniche', 'mensaje': 'Atributo normal'}
Original: Atributo normal

Comprobacion Resultados:
Mensaje original: Atributo normal
Mensaje deserializado: Atributo serializado


### Personalizar deserializacion - Metodo `__setstate__`

Al deserializar un objeto, `pickle` busca una implementacion del metodo `__setstate__`. Se recibe como argumento el diccionario que representa el estado del objeto que fue serializado
- Si **no** esta implementado, se asigna el `__dict__` para la deserialiazcion sin realizar otras acciones

In [32]:
class Perro:

    def __init__(self, nombre, raza) -> None:
        self.nombre = nombre
        self.raza = raza
        self.mensaje = "Atributo normal"

    # implementamos __getstate__

    def __getstate__(self):

        # copiamos el diccionario de metodos y atributos
        nuevo = self.__dict__.copy()

        # modificamos uno de los atributos para la serializacion
        nuevo.update({"mensaje": "Atributo serializado"})

        # lo que se retorna es el resultado de la serializion
        return nuevo
    
    # implementamos __setstate__
    def __setstate__(self, state):
        # implementamos modificaciones
        print('DESERIALIZANDO OBJETO')

        # modificamos un atributo
        state.update({"mensaje": "atributo deserializado"})

        # modificamos el nombre
        state.update({"nombre": "Oddie"})

        # asignamos el dict al nuevo estado deserializado
        self.__dict__ = state
    
    def ladrar(self):
        print('guau')

In [38]:
perro = Perro('TinTin', 'Caniche')
print(perro.__dict__)

print(f'Original: {perro.mensaje}')

# serializamos y deserializamos

perro_ser = pickle.dumps(perro)
perro_deser = pickle.loads(perro_ser)

print('\nComprobacion Resultados:')
print(f'Mensaje original: {perro.mensaje}') # el perro sigue igual
print(f'Mensaje deserializado: {perro_deser.mensaje}')

print(f'Cambio de nombre: {perro_deser.nombre}')

print(f'¿Son iguales? {perro == perro_deser}') # no son identicos, son objetos !=
print(f'¿Son el mismo objeto? {perro is perro_deser}') 

{'nombre': 'TinTin', 'raza': 'Caniche', 'mensaje': 'Atributo normal'}
Original: Atributo normal
DESERIALIZANDO OBJETO

Comprobacion Resultados:
Mensaje original: Atributo normal
Mensaje deserializado: atributo deserializado
Cambio de nombre: Oddie
¿Son iguales? False
¿Son el mismo objeto? False


In [43]:
print(type(perro_deser)) # la deserializacion lo transforma de nuevo a objeto

<class '__main__.Perro'>


`__getstate__` y `__setstate__` son utiles cuando se quiere serializar un objeto, pero que contiene un atributo variable a medida que se ejecuta el codigo (depende de las condiciones actuales del programa)

- En ese caso al realizar la serializacion podriamos resetear valores de ciertas variables a un estado predeterminado


## JSON

- Formato de texto estandar de intercambio de datos, interpretado por muchso lenguajes
- Es *human-readable*

- Por defecto serializa instancias de `int`, `str`, `float`, `dict`, `bool`, `list`, `tuple` y `NoneType`
- Para funciones e instancias de cases es necesario personalizar

### Modulo `json`

- Contiene los metodos `dump`(`s`) y `load`(`s`), igual que `pickle`

In [71]:
import json
from itertools import count

class Gato():
    id = count() # para mantener un contador

    def __init__(self, nombre, edad, raza):
        self.nombre = nombre
        self.edad = edad
        self.raza = raza
        self.id_ = next(self.id)

In [77]:
gato = Gato("Garfield", 4, "Persa")

# lo serializamos como json
gato_json_string = json.dumps(gato.__dict__)

print(f'En formato JSON, tipo: {type(gato_json_string)} -> {gato_json_string}')

print()

# lo deserializamos
gato_json_deserializado = json.loads(gato_json_string)
print(f'En formato Python, tipo: {type(gato_json_deserializado)} -> {gato_json_deserializado}')

# notamos que el formato es muy similar, pero el tipo es distinto


En formato JSON, tipo: <class 'str'> -> {"nombre": "Garfield", "edad": 4, "raza": "Persa", "id_": 5}

En formato Python, tipo: <class 'dict'> -> {'nombre': 'Garfield', 'edad': 4, 'raza': 'Persa', 'id_': 5}


### Personalizar serializacion en JSON - Clase `json.JSONEncoder`

- Para serializar objetos
- Definimos una clase que herede de `json.JSONEncoder` y hacerle *overriding* al metodo `default`. Similar al metodo `__getstate__`

In [84]:
from itertools import count
import json


class Persona:
    id = count()

    def __init__(self, nombre, edad, estado_civil):
        self.nombre = nombre
        self.edad = edad
        self.estado_civil = estado_civil
        self.id_ = next(self.id)


In [85]:
from datetime import datetime

class PersonaEncoder(json.JSONEncoder):

    # efectuamos el overriding
    def default(self, object):
        """Efectuamos una serializacion personalizada"""

        if isinstance(object, Persona):
            return {
                "Persona_id": object.id_,
                "nombre": object.nombre,
                "edad": object.edad,
                "estado_civil": object.estado_civil,
                "ano_nacimiento": datetime.now().year - object.edad,
            }
        
        # en caso de no ser una instancia, mantenemos la serializacion por defecto
        return super().default(object)

In [86]:
p1 = Persona("Juan", 21, 'soltero')

# serializamos de con el metodo por defecto
json_string = json.dumps(p1.__dict__)
print(json_string)

{"nombre": "Juan", "edad": 21, "estado_civil": "soltero", "id_": 0}


In [87]:
# serializacion con nuestra nueva clase
json_string2 = json.dumps(p1, cls=PersonaEncoder)
print(json_string2)

{"Persona_id": 0, "nombre": "Juan", "edad": 21, "estado_civil": "soltero", "ano_nacimiento": 2002}


### Personalizar deserializacion en JSON - Metodo `object_hook`

- analogo a `__setstate__`
- `object_hook` parametro de el/los metodo/s `load`(`s`), se espera una funcion que maneje un diccionario y retorna un objeto de Python

- Todo objeto JSON **es convertido en diccionario de python primero** y luego pasado a funcion argumento de `object_hook` que hara la tranformacion final

In [89]:
def funcion_hook(diccionario):
    return [(key, value) for key, value in diccionario.items()]


# un string en formato json 
json_string = '{"nombre": "Jorge", "edad": 34, "estado_civil": "casado", "puntaje": 90.5}'

# lo deserializamos, el resultado en una lista de Python
datos = json.loads(json_string, object_hook=funcion_hook)

print(datos)

print(type(datos))

[('nombre', 'Jorge'), ('edad', 34), ('estado_civil', 'casado'), ('puntaje', 90.5)]
<class 'list'>


- En object_hook podemos pasar datos (que representan una instancia de una clase) para transformarlos en una clase, por ejemplo

In [93]:
def funcion_hook2(dicc):
    nombre = dicc["nombre"]
    edad = dicc["edad"]
    estado_civil = dicc["estado_civil"]

    return Persona(nombre, edad, estado_civil)

# string en formato JSON que representa los atributos de una instancia de Persona
json_string2 = '{"nombre": "Jorge", "edad": 34, "estado_civil": "casado"}'

persona_deserializada = json.loads(json_string2, object_hook=funcion_hook2)

print(persona_deserializada) # terminamos con un objeto Persona


### JSON solo deja encerrar *string* con *double quotes* (`""`)

Las *single quotes* (`'`) de Python no son valida en JSON, se deben transformar en *double quotes* (`"`) 

Teniendo un diccionario en python, para pasarlo a un string valido JSON se debe hacer
```python
json_str = str(py_dict).replace("\'", "\"")

```

- desventaja: puede ocurrir que existan datos que contengan una ', esta sera reemplazada por una "

# 2. Manejo de Bytes

## Bytes y Encoding

- **Byte**: secuencia de 8 bits
- **Bit**: 0 o 1
- Los computadores almacenan numeros en formado de Bytes
- Para todas las combinaciones entre 0 y 1 en 8 bits, tenemos 2^8 = 256, por lo que con 1 Byte podemos almacenar **un numero en el rango de [0-255]**

**Encoding**: acto de asociar un byte a una lera e interpretar al byte como tal

### Sistema decimal - `chr()` , `ord()`

In [9]:
# chr(), pasa de el byte en codigo decimal a el caracter correspondiente

for i in range(65, 91):
    print(chr(i), end=', ')

A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W, X, Y, Z, 

- `ord(char)` cumple la funcion contraria
```python
ord('B') -> 66
```

### Sistema Hexadecimal - `hex()`

- Base 16, se usan los digitos 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, A, B, C, D, E, F, que representan los valores del 0 al 15

- Para representalos se antepone un `0x`

- `hex()` nos permite obtener la repr hexadecimal de un numero

In [10]:
# Los primeros 16 digitos
for i in range(0, 16):
    print(hex(i), end=', ')

for i in range(0, 16):
    print(f'Decimal: {i}, Hexadecimal: {hex(i)}')

0x0, 0x1, 0x2, 0x3, 0x4, 0x5, 0x6, 0x7, 0x8, 0x9, 0xa, 0xb, 0xc, 0xd, 0xe, 0xf, Decimal: 0, Hexadecimal: 0x0
Decimal: 1, Hexadecimal: 0x1
Decimal: 2, Hexadecimal: 0x2
Decimal: 3, Hexadecimal: 0x3
Decimal: 4, Hexadecimal: 0x4
Decimal: 5, Hexadecimal: 0x5
Decimal: 6, Hexadecimal: 0x6
Decimal: 7, Hexadecimal: 0x7
Decimal: 8, Hexadecimal: 0x8
Decimal: 9, Hexadecimal: 0x9
Decimal: 10, Hexadecimal: 0xa
Decimal: 11, Hexadecimal: 0xb
Decimal: 12, Hexadecimal: 0xc
Decimal: 13, Hexadecimal: 0xd
Decimal: 14, Hexadecimal: 0xe
Decimal: 15, Hexadecimal: 0xf


## objeto `bytes`

- Secuencia **inmutable como los ``str``**
- se declaran con `b"secuencia"`
- Solo podemos usar literales ascii (127 bytes) para la creacion de secuencias de bytes

- los encodea en utf-8

In [59]:
type(b'hola')

bytes

In [20]:
def imprimir_secuencia_bytes(secuencia: bytes):
    print(secuencia)
    print(type(secuencia))

In [21]:
caracteres = b"\x68\x6f\x6c\x61"
imprimir_secuencia_bytes(caracteres)

b'hola'
<class 'bytes'>


- simbolo escape `\x`, indica que los proximos dos caracteres inmediatos representan a un byte en formado ,hexadecimal.  `\x65` es un byte que representa a A

- simbolos que no son reconocidos como ascii, se imprimen como hex

In [24]:
# cliché

caracs = b"\x63\x6c\x69\x63\x68\xe9"
imprimir_secuencia_bytes(caracs)

# xe9 = é, no reconocido como ascii

b'clich\xe9'
<class 'bytes'>


### Ejemplo:
A
- Decimal: 65
- Hexadecimal: 0x41

In [32]:
print(hex(65)) # A en hexadecimal
imprimir_secuencia_bytes(b"\x41") 

0x41
b'A'
<class 'bytes'>


### Decodificacion
Se debe conocer bajo que codificacion fue codificado una serie de bytes. Estos pueden significar distintos simbolos en distintos formatos

- usamos `.decode(<formato_codificacion>)`

In [26]:
# cliché
caracs = b"\x63\x6c\x69\x63\x68\xe9"
print(caracs.decode('latin-1'))
# latin-1 reconoce a xe9 como 'é'

cliché


In [34]:
# Usando distintos formatos
print(caracs.decode('latin-1'))
print(caracs.decode('iso8859-5'))
print(caracs.decode('CP437'))
print(caracs.decode('utf-16'))

cliché
clichщ
clichΘ
汣捩


#### Ejemplo. A y B en Decimal y Hexadecimal

In [47]:
# tambien podemos crear secuencias de bytes usando
# numeros decimales

A_y_B = bytes((65, 66))
print(A_y_B)

# decodicamos estos dos bytes en su formato ascii
print(f'65 y 66 en Ascii: {A_y_B.decode("ascii")}')

# utf-8 es retrocompatible con ascii
print(f'65 y 66 en utf-8: {A_y_B.decode("utf-8")}')

# en uft-16 65 y 66 son cualquier otra cosa
print(f'65 y 66 en utf-16: {A_y_B.decode("utf-16")}')

b'AB'
65 y 66 en Ascii: AB
65 y 66 en utf-8: AB
65 y 66 en utf-16: 䉁


In [42]:
# Mismo ejemplo usando hexadecimales
print(hex(65), hex(66))
A_y_B2 = b"\x41\x42"

print(A_y_B.decode('ascii'))

print(A_y_B.decode('utf-8'))

print(A_y_B.decode('utf-16'))

0x41 0x42
AB
AB
䉁


### Codificacion

- Para codificar un *string*, se usa el metodo `encode`

In [50]:
string = 'hola'

print(string.encode('ascii'))
print(string.encode('utf-8'))
print(string.encode('utf-16'))

b'hola'
b'hola'
b'\xff\xfeh\x00o\x00l\x00a\x00'


Para string que contengan caracteres no dentro de los 127 de *ascii*, se imprimen como hexadecimal

In [52]:
string2 = 'camión'
print(string2.encode('utf-8'))

# como ó no forma parte de ascii, codificar el string tira error
print(string2.encode('ascii'))

b'cami\xc3\xb3n'


UnicodeEncodeError: 'ascii' codec can't encode character '\xf3' in position 4: ordinal not in range(128)

El parametro `errors` nos permite manejar bytes no soportados por algun formato de codificacion

1. `strict`: valor por defecto. Imprime el hex
2. `replace`: reempplaza por un '?'
3. `ignore`: no imprime al caracter
4. `xmlcharrefreplace`: entidad xml que representa al caracter

In [55]:
# 3 formas de codificar en ascii y lidiando con los elementos fuera de los 127

print(string2.encode('ascii', errors='ignore'))
print(string2.encode('ascii', errors='replace'))
print(string2.encode('ascii', errors='xmlcharrefreplace'))

b'camin'
b'cami?n'
b'cami&#243;n'


**Nota**: para codificar strings, irse a la segura con UTF-8

## Objeto `bytearray`

- arreglos de `bytes` **mutables**
- se pueden manipular igual que una lista
- agregamos bytes con el metodo `extend()`

![Alt text](imgs/8_bytearray.png)

![Alt text](imgs/8_byterray2.png)

In [3]:
# se construye con un objeto de bytes inicial
lista_bytes = bytearray(b"helloworld")
print(lista_bytes)

# slicing
print(lista_bytes[5:10])

bytearray(b'helloworld')
bytearray(b'world')


In [4]:
mundo = 'mundo'
for letra in mundo:
    print(hex(ord(letra)))

0x6d
0x75
0x6e
0x64
0x6f


In [5]:
# mutables
# creamos otra secuencia de bytes y la reemplazamos
mundo_bytes = b"\x6d\x75\x6e\x64\x6f"
print(mundo_bytes)

lista_bytes[5:10] = mundo_bytes
print(lista_bytes)

b'mundo'
bytearray(b'hellomundo')


In [95]:
# extensibles
lista_bytes.extend(b"python")
print(lista_bytes)

bytearray(b'hellomundopython')


In [97]:
lista_bytes.extend(b"\x15") # 21 en dec
print(lista_bytes)

bytearray(b'hellomundopython\x15\x15')


### `bin(byte)`
- entrega una representacion binaria del byte

In [7]:
byte1 = lista_bytes[0]
print(f'{byte1} es {type(byte1)}') # literal 'h' es 104 en ascii

byte1 = lista_bytes[0:1]
print(f'{byte1} es {type(byte1)}') # literal 'h' es 104 en ascii

print(bin(byte1)) # 0b indica que esto es binario

104 es <class 'int'>
bytearray(b'h') es <class 'bytearray'>


TypeError: 'bytearray' object cannot be interpreted as an integer

Solo para representacion visual


In [114]:
# para una represetacion en 8bits podemos usar zfill(8)
# esto rellena con 0s hasta llegar a un largo de 8
print(bin(byte1).zfill(0))
print(bin(byte1)[2:].zfill(0)) # para quitarnos los 0b

0b1101000
1101000


In [127]:
# podemos definir una funcion para simplificar
binario = lambda byte: bin(byte)[2:].zfill(8)
hexadecimal = lambda byte: hex(byte)[2:]

for i in range(0, 20):
    print(f'Decimal: {i}')
    print(f'Binario: {binario(i)}')
    print(f'Hexadecimal: {hexadecimal(i)}')
    print()

Decimal: 0
Binario: 00000000
Hexadecimal: 0

Decimal: 1
Binario: 00000001
Hexadecimal: 1

Decimal: 2
Binario: 00000010
Hexadecimal: 2

Decimal: 3
Binario: 00000011
Hexadecimal: 3

Decimal: 4
Binario: 00000100
Hexadecimal: 4

Decimal: 5
Binario: 00000101
Hexadecimal: 5

Decimal: 6
Binario: 00000110
Hexadecimal: 6

Decimal: 7
Binario: 00000111
Hexadecimal: 7

Decimal: 8
Binario: 00001000
Hexadecimal: 8

Decimal: 9
Binario: 00001001
Hexadecimal: 9

Decimal: 10
Binario: 00001010
Hexadecimal: a

Decimal: 11
Binario: 00001011
Hexadecimal: b

Decimal: 12
Binario: 00001100
Hexadecimal: c

Decimal: 13
Binario: 00001101
Hexadecimal: d

Decimal: 14
Binario: 00001110
Hexadecimal: e

Decimal: 15
Binario: 00001111
Hexadecimal: f

Decimal: 16
Binario: 00010000
Hexadecimal: 10

Decimal: 17
Binario: 00010001
Hexadecimal: 11

Decimal: 18
Binario: 00010010
Hexadecimal: 12

Decimal: 19
Binario: 00010011
Hexadecimal: 13



- imprimir un elemento de un bytearray nos muestra el int que representa, `ord` hace esta conversion directamente

### `bytearrays` como listas

In [11]:
mi_ba = bytearray()

# append solo funciona con ints
mi_ba.append(255)
print(mi_ba)

# extend funciona con byte o bytearray
mi_ba.extend(b'\x02') # bytes
print(mi_ba)

mi_ba.extend(bytearray(b'\x34\x12'))
print(mi_ba)

bytearray(b'\xff')
bytearray(b'\xff\x02')
bytearray(b'\xff\x024\x12')


In [13]:
mi_ba[0] # indexing muestra el valor int

for i in mi_ba:
    print(i)

255
2
52
18


In [15]:
# solicing muestra los bytes
print(mi_ba[:2])

bytearray(b'\xff\x02')


#### Maximo y Minimos en un bytearray

In [19]:
maximo = max(mi_ba)
minimo = min(mi_ba)

print(maximo)
print(minimo)

255
2


## Chunks

- **Chunks**: grupo de bytes
- es util leer grupos de bytes de una sola vez

In [29]:
array_grande = bytearray(b"Este es un arreglo muy largo que esta codificado en bits")
TAMANO_CHUNK = 4

for i in range(0, len(array_grande), TAMANO_CHUNK):
    chunk = bytearray(array_grande[i:i+TAMANO_CHUNK])
    print(chunk)

bytearray(b'Este')
bytearray(b' es ')
bytearray(b'un a')
bytearray(b'rreg')
bytearray(b'lo m')
bytearray(b'uy l')
bytearray(b'argo')
bytearray(b' que')
bytearray(b' est')
bytearray(b'a co')
bytearray(b'difi')
bytearray(b'cado')
bytearray(b' en ')
bytearray(b'bits')


## Metodos de `int` para interpretar *bytes*

Con un solo *byte* podemos almacenar desde el 0 (`0x00`) hasta el 255 (`0xff`) . Para representar numeros mas grandes es cosa de añadir *bytes*

El problema surge en como interpretar el orden de los bytes: `byteorder`


* En *big endian* el *byte* más significativo (de mayor peso) quedará al inicio del *byte array*. Por ejemplo, el número 256 en *big endian* es `\x01 \x00`.

`hex()` da numeros en *big endian*

* En *little endian* es lo opuesto, el *byte* más significativo (de mayor peso) quedará al final del *byte array*. Por ejemplo, el número 256 en *little endian* es `\x00 \x01`.

### `int.to_bytes`

- `int.to_bytes(length = 1, byteorder = 'big')`: retorna el `int` representado por un arreglo de bytes. Recibe el largo del array y el orden (`big` o `little`)

    Si `length` es mas largo que la cantidad necesaria de *bytes* para representar el numero, se rellenaran con 0 de modo que el valor no cambie

In [52]:
for i in range(65, 99):
    print((i).to_bytes(1, 'big'), end=' - ')

b'A' - b'B' - b'C' - b'D' - b'E' - b'F' - b'G' - b'H' - b'I' - b'J' - b'K' - b'L' - b'M' - b'N' - b'O' - b'P' - b'Q' - b'R' - b'S' - b'T' - b'U' - b'V' - b'W' - b'X' - b'Y' - b'Z' - b'[' - b'\\' - b']' - b'^' - b'_' - b'`' - b'a' - b'b' - 

In [49]:
(255).to_bytes(4, 'big') # 000255

b'\x00\x00\x00\xff'

In [43]:
(256).to_bytes(2, 'big')

b'\x01\x00'

In [46]:
(256).to_bytes(2, 'little')

b'\x00\x01'

In [47]:
(256).to_bytes(1, 'big') # no podemos expresar a 256 en 1 byte

OverflowError: int too big to convert

### `int.from_bytes`

* `int.from_bytes(byte, byteorder = 'big')`: retorna un arreglo de *bytes* representando un *integer*. Debe recibir al menos dos argumentos: el *byte* (cualquier objeto del tipo byte o un un iterable que produzca *bytes*) y el orden. Si `byteorder` es 'big', el *byte* mas significativo (de mayor peso) será el del inicio del *byte array*. Si  `byteorder` es 'little', el byte mas significativo (de mayor peso) será el del final del *byte array*.

    es como el inverso de ``hex()``

In [72]:
# cambiar el byteorder para mas de un byte, se cambia radicalmente el orden

print(int.from_bytes(b'A', byteorder='big')) # a representa al literal 65
print(int.from_bytes(b'AA', byteorder='little'))

65
16705


In [65]:
# dos bytes representando al 256

print(int.from_bytes(b'\x01\x00', byteorder='big'))
print(int.from_bytes(b'\x01\x00', byteorder='little'))

256
1


In [70]:
print(int.from_bytes(b'\x01\x00\x00', byteorder='big'))

65536


- No confiar en los prints de *bytes*, `print()` solo nos entrega una representacion visual. Se debe verificar el byte con con el metodo `to_bytes`

In [80]:
# \x41 es 65

# comprobacion
int.from_bytes(b'\x41', byteorder='big')

65

In [83]:
# pero realizando el contrario...
(65).to_bytes(1, 'big')
# ... nos da la representacion literal

b'A'

In [84]:
# podemos incluso meter la representacion visual e interpretarla como int
int.from_bytes(b'A', 'big')

65